In [1]:
from langchain.memory import ConversationBufferMemory

In [7]:
memory = ConversationBufferMemory(return_messages=True)

In [10]:
memory.save_context({"input":"Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [11]:
memory.save_context({"input":"Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}